<a href="https://colab.research.google.com/github/artanebibi/datascience/blob/main/Neural_Networks_and_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense

In [2]:
!gdown 10xd_Zk0Ld_GlonTOFLNpuLyThOyi-hvq

Downloading...
From (original): https://drive.google.com/uc?id=10xd_Zk0Ld_GlonTOFLNpuLyThOyi-hvq
From (redirected): https://drive.google.com/uc?id=10xd_Zk0Ld_GlonTOFLNpuLyThOyi-hvq&confirm=t&uuid=68904048-7c85-447b-9a04-e19265f663de
To: /content/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv
100% 318M/318M [00:03<00:00, 90.6MB/s]


In [3]:
df = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')

In [4]:
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit = 's')

In [6]:
df.set_index(df['Timestamp'])

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,,
2011-12-31 07:52:00,2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
2011-12-31 07:53:00,2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2021-03-30 23:56:00,2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
2021-03-30 23:57:00,2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
2021-03-30 23:58:00,2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496


In [7]:
df.drop(columns = 'Timestamp', inplace = True)

In [ ]:
df.ffill(inplace=True).bfill(inplace=True)

In [8]:
df.sort_index(inplace = True) # sort by time because of time-series data

In [9]:
copy = df[['Close']].copy() # create a copy of the df

In [10]:
lag = 3

In [11]:
periods = range(lag, 0, -1)

In [12]:
copy = copy.shift(periods=periods) # introduce

In [13]:
copy.dropna(axis = 0, inplace = True)

In [14]:
copy

,Close_3,Close_2,Close_1
13217,6.90,6.90,6.90
13218,6.90,6.90,6.90
13226,6.50,6.50,6.50
13227,6.50,6.50,6.50
13228,6.50,6.50,6.50
...,...,...,...
4857372,58698.50,58739.95,58714.31
4857373,58739.95,58714.31,58686.00
4857374,58714.31,58686.00,58685.81
4857375,58686.00,58685.81,58723.84


In [15]:
df = pd.concat([df, copy], axis = 1)

In [16]:
df

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Close_3,Close_2,Close_1
0,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4857372,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339,58698.50,58739.95,58714.31
4857373,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508,58739.95,58714.31,58686.00
4857374,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496,58714.31,58686.00,58685.81
4857375,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202,58686.00,58685.81,58723.84


In [17]:
df.dropna()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Close_3,Close_2,Close_1
13217,6.90,6.90,6.90,6.90,0.699259,4.824886,6.900000,6.90,6.90,6.90
13226,6.50,6.50,6.50,6.50,1.557632,10.124611,6.500000,6.50,6.50,6.50
13227,6.50,6.50,6.50,6.50,1.564945,10.172144,6.500000,6.50,6.50,6.50
13228,6.50,6.50,6.50,6.50,1.545595,10.046368,6.500000,6.50,6.50,6.50
14527,6.24,6.24,6.24,6.24,1.602564,10.000000,6.240000,6.24,6.24,6.24
...,...,...,...,...,...,...,...,...,...,...
4857372,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339,58698.50,58739.95,58714.31
4857373,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508,58739.95,58714.31,58686.00
4857374,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496,58714.31,58686.00,58685.81
4857375,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202,58686.00,58685.81,58723.84


In [18]:
X, y = df.drop(columns=["Close"]), df["Close"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=False)

In [19]:
scaler = MinMaxScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [20]:
scaler = MinMaxScaler()
train_y = scaler.fit_transform(train_y.to_numpy().reshape(-1, 1))

In [21]:
train_y.shape

(3885901, 1)

In [22]:
train_X.shape


(3885901, 9)

In [23]:
(train_X.shape[0], lag, (train_X.shape[1] // lag)) # (samples, timesteps, features)

(3885901, 3, 3)

In [24]:
train_X = train_X.reshape(train_X.shape[0], lag, (train_X.shape[1] // lag))
test_X = test_X.reshape(test_X.shape[0], lag, (test_X.shape[1] // lag))

LSTM's EXPECT (samples, timesteps, features) SHAPE

In [25]:
model = Sequential([
    Input((train_X.shape[1], train_X.shape[2],)),
    LSTM(8, activation="relu", return_sequences=True),
    LSTM(4, activation="relu"),
    Dense(1, activation="linear")
])


In [26]:
model.compile(
    loss="mean_squared_error",
    optimizer="adam",
    metrics=["mean_squared_error"],
)

In [ ]:
history = model.fit(train_X, train_y, validation_split=0.2, epochs=4, batch_size=16, shuffle=False)

Epoch 1/4
108542/194295 ━━━━━━━━━━━━━━━━━━━━ 4:00 3ms/step - loss: nan - mean_squared_error: nan